Merger File

In [130]:
import os
import regex as re
import pandas as pd
import numpy as np

In [6]:
# import episodes dataframe and take a look at it 
sponge_df = pd.read_csv('sponge_df.csv')

# replace Krabs á la Mode...having trouble with accent in text file
sponge_df.iloc[sponge_df[sponge_df.Title.str.contains('Krabs á')].index[0],1] = \
'Krabs a la Mode'


sponge_df.head(10)

,Episode #,Title,Rating
0,0,Help Wanted,8.7
1,1,Reef Blower,8.7
2,2,Tea at the Treedome,8.7
3,3,BubbleStand,8.5
4,4,Ripped Pants,8.5
5,5,Jellyfishing,8.4
6,6,Plankton!,8.4
7,7,Naughty Nautical Neighbors,8.5
8,8,Boating School,8.5
9,9,Pizza Delivery,9.1


In [107]:
sponge_df.iloc[sponge_df[sponge_df.Title.str.contains('Krabs á')].index[0],1] = \
'Krabs a la Mode'

In [109]:
#sponge_df[sponge_df.Title.str.contains('Krabs á')]['Title'] = 'Krabs a la Mode'
sponge_df[sponge_df.Title.str.contains('Krabs')]

,Episode #,Title,Rating
47,47,Imitation Krabs,9.2
107,107,Born Again Krabs,8.6
120,120,Krabs vs. Plankton,8.1
166,166,Krabs a la Mode,7.9
277,277,Kracked Krabs,5.8
316,316,Mr. Krabs Takes a Vacation,6.6
353,353,Safe Deposit Krabs,7.6
362,362,Yeti Krabs,6.6
492,492,My Two Krabses,7.1


### assess similarities between "title" in dataframe and title in scripts. want 393 matches in sponge df need to strip punctuation and strings modified scrape edge cases below until cell below shows 393 matches
1. InSPONGEiac.txt to InSpongeiac imdb
1. Karen.txt vs Karen 2.0 imdb  -- changed txt to 20
1. SB.txt vs SB-129 imdb -- changed txt
1. ASquarePantsFamilyVacation.txt vs SquarePants Family Vacation
1. Ugh.txt vs SpongeBob B.C. (Before Comedy) (Ugh) --> spongebobbcbeforecomedyugh.txt
1. PattyCaper.txt vs The Patty Caper
1. GaryampSpot.txt vs Gary&Spot -- changed text to GarySpot
1. PartyPooperPants.txt vs SpongeBob's House Party(Party Pooper Pants)
changed the txt file name to SpongeBobsHousePartyPartyPooperPants.txt
1. ForHereToGo vs For Here or to Go imdb -- changed text to include "or"
1. Krabslamode.txt vs Krabs á la Mode imdb. accent issue, will switch df accent
1. Whatever Happened to Spongebob vs Whatever Happened to SpongeBob? (WhoBob WhatPants)
ended up changing txt file
1. Trenchbilles.txt vs Trenchbillies imdb -- changed txt
1. pattiesunderthesea.txt vs 20,000 patties under the sea --> add 20000 to start of txt name
1. YouDontKnowSpongebob.txt vs You Don't Know Sponge imdb --> change text file bc real name is imdb

In [118]:

 #get lowercase text file names -- they're already stripped
text_files_names = [
    file.replace('.txt','').lower() for file in os.listdir('transcripts')
    ]

# get lowercase and stripped titles from the dataframe
lower_df_titles = [
    re.sub('\W+','', x.lower()) for x in list(sponge_df.Title)
    ]

In [119]:
print('In our matchup, we have found {} matches out of 393 total.'
    .format(
    len(set(text_files_names).intersection(set(lower_df_titles))))
    )

In our matchup, we have found 393 matches out of 393 total.


In [120]:
# if empty, we're happy
list(set(text_files_names).difference(set(lower_df_titles)))

[]

In [125]:
# method of checking dataframe to see episode names
sponge_df[sponge_df.Title.str.contains("Krabs")]

,Episode #,Title,Rating
47,47,Imitation Krabs,9.2
107,107,Born Again Krabs,8.6
120,120,Krabs vs. Plankton,8.1
166,166,Krabs a la Mode,7.9
277,277,Kracked Krabs,5.8
316,316,Mr. Krabs Takes a Vacation,6.6
353,353,Safe Deposit Krabs,7.6
362,362,Yeti Krabs,6.6
492,492,My Two Krabses,7.1


First, let's align each text file in `text_files_names` with `lower_df_titles` that way we can figure out where to map each script. The normal list of the filenames should have the same index as the one which was modified, so we can locate the index of the filename that way and should be able to call it from `os`.

In [136]:
# get normal text file names 
text_files_names_normal =  [
    file for file in os.listdir('transcripts')
    ]

In [222]:
# now let's just keep the episodes from the text files! 
sponge_df = sponge_df.query(
    "Title.str.replace(r'\W+', '', regex=True).str.lower() in @text_files_names"
    )
sponge_df

,Episode #,Title,Rating
0,0,Help Wanted,8.7
1,1,Reef Blower,8.7
2,2,Tea at the Treedome,8.7
4,4,Ripped Pants,8.5
6,6,Plankton!,8.4
...,...,...,...
494,494,The Hankering,6.7
497,497,Who R Zoo?,7.1
500,500,Biddy Sitting,7.1
501,501,Lighthouse Louie,6.8


Now that we have verified matches, let's create a link between imdb and the scripts, and add the scripts into the dataframe. 

In [201]:
# Necessary regex to get rid of actions in brackets and parentheses
# space pattern will get rid of the remainder of the action lines if that made it empty
parentheses_pattern = re.compile(r'\(.*\)')
brackets_pattern = re.compile(r'\[.*\]')
space_pattern = re.compile(r'^.*:\s$')
just_period_pattern = re.compile(r'^.*:\s\.$')



# create get script function
def get_script(imdb_title):

    title = re.sub('\W+','', imdb_title.lower())
    index = text_files_names.index(title)
    filename = text_files_names_normal[index]
    
    path = os.path.join('transcripts', filename)
    f = open(path,"r").read()
    lines = f.split("\n")

    for i in range(len(lines)):
        # remove brackets
        if re.search(brackets_pattern, lines[i]):
            lines[i] = re.sub(brackets_pattern, '', lines[i])
        # remove parentheses
        if re.search(parentheses_pattern, lines[i]):
            lines[i] = re.sub(parentheses_pattern, '', lines[i])
        # remove empty lines from characters
        if re.search(space_pattern, lines[i]):
            lines[i] = re.sub(space_pattern, '', lines[i])
        if re.search(just_period_pattern, lines[i]):
            lines[i] = re.sub(just_period_pattern, '', lines[i])
        
    # remove empty strings -- some lines are just actions
    lines = [i for i in lines if i]
    # actually, some are spaces. let's get rid of those too 
    lines = [i for i in lines if i != ' '] 

    return lines


    

In [223]:
# add script to each episode
sponge_df['Script'] = sponge_df.Title.apply(lambda x: get_script(x))

<ipython-input-223-880fdd59675d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sponge_df['Script'] = sponge_df.Title.apply(lambda x: get_script(x))


In [225]:
sponge_df.head()

,Episode #,Title,Rating,Script
0,0,Help Wanted,8.7,"[French Narrator: Ah, the sea... so fascinatin..."
1,1,Reef Blower,8.7,[]
2,2,Tea at the Treedome,8.7,"[Sandy Cheeks: Take that, you sorry old clam! ..."
4,4,Ripped Pants,8.5,"[Narrator: Ah, Goo. Lagoon. A stinky mud puddl..."
6,6,Plankton!,8.4,"[Squidward: Come on, SpongeBob!, SpongeBob: Ne..."


Let's drop the episode `Reef Blower` because it is an episode without words, it's just actions, so we can't analyze any talking here.

In [228]:
sponge_df = sponge_df.query("Title != 'Reef Blower'")
sponge_df.head()

,Episode #,Title,Rating,Script
0,0,Help Wanted,8.7,"[French Narrator: Ah, the sea... so fascinatin..."
2,2,Tea at the Treedome,8.7,"[Sandy Cheeks: Take that, you sorry old clam! ..."
4,4,Ripped Pants,8.5,"[Narrator: Ah, Goo. Lagoon. A stinky mud puddl..."
6,6,Plankton!,8.4,"[Squidward: Come on, SpongeBob!, SpongeBob: Ne..."
7,7,Naughty Nautical Neighbors,8.5,"[Squidward: Wow. Squidward, this is the best s..."


In [235]:
sponge_df.to_csv('sponge_df_script.csv')

In [ ]:
# when loading do this 
# sponge_df = pd.read_csv('sponge_df_script.csv',index_col= 0, converters={'Script': literal_eval})